In [1]:
from classes.Tronaduras_File_Reader_v03 import TronadurasFileReader
from classes.Sismicidad_File_Reader import SismicidadFileReader
from classes.DuckDB_Helper_v02 import DuckDBHelper 
import pandas as pd
import pickle
import ipywidgets as widgets
from IPython.display import display
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os


#╭──────────────────────────────────────────────────────╮
#| Preprocesamiento de datos                            | 
#╰──────────────────────────────────────────────────────╯
# 📌 Nombre de la base de datos DuckDB
DB_NAME = "data/Tronaduras_vs_Sismicidad.db"
SCHEMA_NAME = "Raw_Data"
CONJUNTO_ESPECIFICO = "Tronaduras"


# Instanciar las clases de lectura de archivos
Sismicidad_Reader = SismicidadFileReader()
Tronaduras_Reader = TronadurasFileReader()

# Procesar el archivo de sismicidad
df_sismicidad = Sismicidad_Reader.read_file()


#╭─────────────────────────────────────────────────────────────────────────────────────────────╮
#| Conexión a la base de datos y obtención de listado de variables de tronaduras a leer        |
#╰─────────────────────────────────────────────────────────────────────────────────────────────╯

# 🔗 Instanciar y conectar
db_helper = DuckDBHelper(DB_NAME)

# 🔍 Obtener que variables de tronaduras serán leídas
tronaduras_variables_df = db_helper.select_df(
    table="Variables_Description",
    columns='"Variable", "Lectura", "Tipo"',  # 🔥 Selecciona solo las columnas necesarias
    where=f'Conjunto = \'{CONJUNTO_ESPECIFICO}\'',  # 🔥 Filtra por conjunto
    schema=SCHEMA_NAME
)

# 🚪 Cerrar conexión
# db_helper.close_connection()


# Procesar el archivo de tronaduras
df_tronaduras = Tronaduras_Reader.read_file(
    file_name="Informacion_Guitarra_TC_P0_Zublin_312_Limpiav2.xlsx", 
    sheet_index=2,
    directory="../Datos Teniente/",
    variables_df=tronaduras_variables_df
)

# Aplicar la función a la columna GSI_Puntaje
df_tronaduras["GSI_Puntaje"] = df_tronaduras["GSI_Puntaje"].apply(Tronaduras_Reader.mean_range)



#╭────────────────────────────────────────────────────────╮
#| Guardar Tablas en DuckDB                               |
#╰────────────────────────────────────────────────────────╯
# Crear las tablas en la base de datos a partir de los DataFrames:
db_helper.create_table_from_df("Tronaduras", df_tronaduras, schema="Raw_Data")
db_helper.create_table_from_df("Sismicidad", df_sismicidad, schema="Raw_Data")

# Cierre de la conexión
db_helper.close_connection()

🔗 Conexión establecida con data/Tronaduras_vs_Sismicidad.db
✅ Datos leídos desde 'Raw_Data.Variables_Description' y convertidos a DataFrame.
⚠️ Usando hoja por defecto: Base Datos Zublin 
✅ Columnas renombradas: 5
✨ Columnas encontradas en el archivo: ['N° Disparo', 'Fecha', 'Hora', 'PK', 'Cobertura Total', 'Cobertura  Primario', 'Tronadura_Largo de Perforación (m)', 'Tronadura_N° Tiros', 'Tronadura_N° Tiros Real', 'Tronadura_Kg. de explosivos tronadura', 'Tronadura_Tipo Explosivo', 'Destressing_Se realizó', 'Destressing_N° Tiros', 'Destressing_Kg. de explosivos', 'Destressing_Tipo Explosivo', 'Auscultacion_Desde (m)', 'Auscultacion_Hasta (m)', 'Auscultacion_Observación', 'Sondajes_Desde (m)', 'Sondajes_Hasta (m)', 'Sondajes_Estructura', 'Geotecnicas_UCS (MPa)', 'Geotecnicas_Modulo de Young (GPa)', 'Geotecnicas_Razón Poisson', 'Litología', 'Fallas_Presencia', 'Fallas_Sub-Paralelas', 'Estructura', 'Estructura Crítica', 'Condición de Agua', 'GSI_Puntaje', 'GSI_Blocosidad', 'GSI_Condición